In [1]:
from langchain.document_loaders import TextLoader

In [2]:
file_path = r"C:\Users\prana\Downloads\Aditi\Pranav\Portfolio_Projects\News_article.txt"
loader = TextLoader(file_path)
data = loader.load()
data

[Document(page_content='The Aditya Birla Group, with seven listed firms in its fold, added around Rs 1.91 lakh crore in market capitalisation in the last year. In terms of market capitalisation, it currently ranks as the seventh largest conglomerate in the country.\n\nHindalco, the second largest company in the group is the news because of its subsidiary Novelis filing for an initial public offering in the US. Hindalco, is on a path to recovery, with global growth looking up, spelling good news for its overseas subsidiary Novelis, although the latterâ€™s capex guidance has met with resistance from the street.\n\nThe cement business has been the crown jewel of the group and continued to lead the show. Notwithstanding, the entry of a large corporate group, the stock did well thanks to buoyant demand coming from post-COVID recovery, boosted by government spending and housing revival. Operating margins also got a boost from benign input prices. While demand revival might pause due to upcom

In [35]:
text = "The Aditya Birla Group, with seven listed firms in its fold, added around Rs 1.91 lakh crore in market capitalisation in the last year. In terms of market capitalisation, it currently ranks as the seventh largest conglomerate in the country.\n\nHindalco, the second largest company in the group is the news because of its subsidiary Novelis filing for an initial public offering in the US. Hindalco, is on a path to recovery, with global growth looking up, spelling good news for its overseas subsidiary Novelis, although the latterâ€™s capex guidance has met with resistance from the street.\n\nThe cement business has been the crown jewel of the group and continued to lead the show. Notwithstanding, the entry of a large corporate group, the stock did well thanks to buoyant demand coming from post-COVID recovery, boosted by government spending and housing revival. Operating margins also got a boost from benign input prices. While demand revival might pause due to upcoming elections, the long-term trajectory remains strong and Ultratechâ€™s pan-India operations and efficiency should stand in good stead.\n\nVodafone, which has been facing business challenges with steady erosion in its customer base, got a sentiment boost with recent news reports suggesting a potential equity infusion by the promoters.\n\nAditya Birla Fashion hasnâ€™t had the best of times owing to twin challenges of subdued urban sentiment and the task of improving the profitability of its numerous acquisitions. Aditya Birla Capital, in contrast, has had a strong show with the business growth accelerating under the new CEO. However, given the cautious sentiment towards unsecured lending, growth in the financing business could moderate."

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
r_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n", " ", "."],
    chunk_size = 200,
    chunk_overlap = 0
)
chunks = r_splitter.split_text(text)
len(chunks)

11

In [29]:
for chunk in chunks:
    print(len(chunk))

196
44
199
147
195
198
121
197
199
195
9


In [ ]:
!pip install faiss-cpu
!pip install sentence-transformers

In [ ]:
!pip install streamlit

In [41]:
import os

import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [42]:
os.environ['OPENAI_API_KEY'] = 'API_key'

In [ ]:
LLM = OpenAI(temperature = 0.9, max_tokens = 500)

file_path = r"C:\Users\prana\Downloads\Aditi\Pranav\Portfolio_Projects\News_article.txt"
loader = TextLoader(file_path)
data = loader.load()
data

In [45]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 200
)
docs = text_splitter.split_documents(data)
len(docs)

4

In [46]:
docs[0]

Document(page_content='The Aditya Birla Group, with seven listed firms in its fold, added around Rs 1.91 lakh crore in market capitalisation in the last year. In terms of market capitalisation, it currently ranks as the seventh largest conglomerate in the country.\n\nHindalco, the second largest company in the group is the news because of its subsidiary Novelis filing for an initial public offering in the US. Hindalco, is on a path to recovery, with global growth looking up, spelling good news for its overseas subsidiary Novelis, although the latterâ€™s capex guidance has met with resistance from the street.', metadata={'source': 'C:\\Users\\prana\\Downloads\\Aditi\\Pranav\\Portfolio_Projects\\News_article.txt'})

In [ ]:
!pip install tiktoken

In [ ]:
embeddings = OpenAIEmbeddings()
vectorindex_openai = FAISS.from_documents(docs, embeddings)

In [ ]:
batch_size = 100  # Adjust batch size as needed
embeddings = OpenAIEmbeddings()
vectorindex_openai = FAISS.from_documents(docs, embeddings, batch_size)

In [ ]:
file_path="vector_index.pk1"

with open(file_path, "wb") as f: 
    pickle.dump(vectorindex_openai, f)

In [ ]:
if os.path.exists(file_path):
    with open(file_path, "rb") as f: 
        vectorIndex = pickle.load(f)

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm = llm, retriever = vectorIndex.as_retriever())
chain

In [ ]:
query = "How much capital was added by Aditya Birla Group last year?"

langchain.debug = True
chain({"question": query}, return_only_outputs = True)